# Belegaufgabe zum Blockkurs Python
## Simulation mit vtk
Ziel ist es ein Modul zu schreiben, welches eine Windmühle simuliert. Die Windmühle soll dabei nur
aus primitiven Objekten zusammengesetzt werden und einen Rotor besitzen der sich während der
Simulation dreht. Außerdem soll eine kleine GUI geschrieben werden (mit PyQt), worüber sich die
Simulation starten lässt und sich entsprechende Parameter (Rotationsgeschwindigkeit, Höhe, etc.)
einstellen lassen! Genutzt werden sollen die in der Übung vorgestellten vtk-Objektwrapperklassen!

------
The aim is to write a module that simulates a windmill. The windmill should only be made up of primitive objects and have a rotor that rotates during the simulation. A small GUI should also be written (with PyQt) through which the simulation can be started and the relevant parameters (rotation speed, height, etc.) can be set! The vtk object wrapper classes presented in the exercise should be used!



# Todo
- [ ] class for objects
- [ ] assemle windmill
- [ ] add GUI
- [ ] vtk ???
- [ ] what are wrappers?

In [8]:

material_dict = {}

def add_material(name: str, material_properties: tuple):
    if name not in material_dict:
        material_dict[name] = material_properties
    else: 
        raise Exception("Such material already exists")
    
def get_material_list():
    return material_dict.keys()


class Structure:
    def __init__(self, name: str, mass: float, material: str, *dimensions: float):
        self._name = name
        self._mass = mass
        if material not in material_dict:
            raise Exception("No such material known")
        else:
            self._material = material
        self._dim = []
        if len(dimensions) != 2:
            raise ValueError("Need exactly 2 dimensions")
        else:
            self._dim = sorted(dimensions)

    def get_dimensions(self):
        return self._dim

    def __str__(self):
        return "This is a %s %s of %d kg mass, made of %s. The size is %dx%d" % (self.__class__.__name__, self._name, self._mass, self._material, self._dim[0], self._dim[1])

structure_list = []

class Rotor(Structure):
    def __init__(self,  name: str, mass: float, material: str, max_rpm: float, model: str,  efficiency: float, *dimensions: float):
        super().__init__(name, mass, material, *dimensions)
        self._max_rpm = max_rpm
        self._model = model
        self._efficiency = efficiency

rotor_list = []

class Wing(Structure):
        def __init__(self,  name: str, mass: float, material: str, number_of_wings: int, *dimensions: float):
            super().__init__(name, mass, material, *dimensions)
            self._now = number_of_wings

wing_list = []



There are objects:

- Fundament (name, mass, width, length, material)
- Tube (name, mass, diameter, heigth, material)
- Rotor (name, mass, max_rpm, width=length, height, efficiency, model)
- Wing (name, mass, number_of_wings, material, length, width)
---
therefore 
- class Structure(name, mass, dimensions, material) can be used for everything ->
- Rotor is an ingeritage of sructure with max_rpm, model and efficiency parameter
- Wing is an ingeritage of structure with number_of_wings parameter

In [9]:
add_material("Steel", ("metallic", 600, 90))  # colour, how strong it is, how it heats
add_material("Wood", ("brown", 100, 30))
add_material("Glass", ("transparent", 20, 40))
add_material("Concrete", ("grey", 200, 20))

print(get_material_list())


Fundament1 = Structure("Fundament", 1000.0, "Concrete", 20, 30)
structure_list.append(Fundament)

print(Fundament)
    
Rotor1 = Rotor("Rotor", 420, "Steel", 1400, "mk. 1", 0.6, 40, 30)   
rotor_list.append(Fundament)

print(Rotor1) 

Tube1 = Structure("Tube", 650.0, "Steel", 2, 60)
structure_list.append(Tube1)

print(Tube1)

Wing1 = Wing("Wing 3x", 80, "Steel", 3, 16, 2)
wing_list.append(Wing1)

print(Wing1)

dict_keys(['Steel', 'Wood', 'Glass', 'Concrete'])
This is a Structure Fundament of 1000 kg mass, made of Concrete. The size is 20x30
This is a Rotor Rotor of 420 kg mass, made of Steel. The size is 30x40
This is a Structure Tube of 650 kg mass, made of Steel. The size is 2x60
This is a Wing Wing 3x of 80 kg mass, made of Steel. The size is 2x16


In [6]:
#import vtk
import vtkmodules.vtkInteractionStyle
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkFiltersSources import vtkCylinderSource
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer
)


def main():
    colors = vtkNamedColors()

    # Create a cylinder
    cylinderSource = vtkCylinderSource()
    cylinderSource.SetCenter(0.0, 0.0, 0.0)
    cylinderSource.SetRadius(5.0)
    cylinderSource.SetHeight(7.0)
    cylinderSource.SetResolution(100)

    # Create a mapper and actor
    mapper = vtkPolyDataMapper()
    mapper.SetInputConnection(cylinderSource.GetOutputPort())
    actor = vtkActor()
    actor.GetProperty().SetColor(colors.GetColor3d('Cornsilk'))
    actor.SetMapper(mapper)

    # Create a renderer, render window, and interactor
    renderer = vtkRenderer()
    renderWindow = vtkRenderWindow()
    renderWindow.SetWindowName('Cylinder')
    renderWindow.AddRenderer(renderer)
    renderWindowInteractor = vtkRenderWindowInteractor()
    renderWindowInteractor.SetRenderWindow(renderWindow)

    # Add the actor to the scene
    renderer.AddActor(actor)
    renderer.SetBackground(colors.GetColor3d('DarkGreen'))

    # Render and interact
    renderWindow.Render()
    renderWindowInteractor.Start()


if __name__ == '__main__':
    main()

Coordiatetes are always ```x, y, z```, where x-y in horisontal and z is vertical for normal state, therefore:
- For Fundament coordinates of the center should be ```0, 0, -0.05*tube._dim[1]``` and dimensions respectively ```dim[1]//2, dim[0]//2, 0.1 * tube._dim[1]```: W, L predefined, H = .1 of tube's H.
- Then the top surface of fundament shoud be ```z=0``` and it will be centered over (0, 0) (x,y)-wise
-----
- For Tube coordinates of the center are ```(0, 0, tube._dim[1]*0.5)```. Radius is tube._dim[0]*0.5 and Height = tube._dim[1]
-----
- For Rotor coordinates of the center are ```(0, 0, tube._dim[1] + rotor.dim[0]//2)``` and dimensions rotor.dim[1]x rotor.dim[1] x rotor.dim[0] (it is square 10 x 10 and 8 in height for example)
-----
- For Fan (which is wings * number of wings) center is ```(0, rotor.dim[1]//2,  tube._dim[1] + rotor.dim[0]//2 + wing.dim[0]//2)``` -- z is the same, x is also 0, y is moved by a half thickness of rotor and a half thickness of wing. Dimensions No idea

In [14]:
class Windmill():
    def __init__(self, fund: Structure, tube: Structure, rotor: Rotor, wing: Wing):
        self._fund = fund
        self._tube = tube
        self._wing = wing
        self._rotor = rotor
    
    def assemble(self):
        tube_r, tube_h = self._tube.get_dimensions()
        fund_l, fund_w = self._fund.get_dimensions()
        fund_h= tube_h * 0.1 
        rotor_h, rotor_w = self._rotor.get_dimensions()  # L is x, W is y, H is z
        rotor_l = rotor_w
        wing_w, wing_l = self._wing.get_dimensions()
        
        self._fund_center = (0, 0, -0.5 * fund_h)
        self._tube_center = (0, 0, tube_h * 0.5)
        self._rotor_center = (0, 0, tube_h * 0.5 + rotor_h * 0.5)
        self._fan_center = (0, rotor_w * 0.5 + wing_w * 0.5, tube_h * 0.5 + rotor_h * 0.5)
        
        pass
        #return vtk object (or list of objects) that does smth

    def render(self):
        pass

    def change_rotation_speed(self):
        pass

    def change_part(self):
        pass

Windmill1 = Windmill(Fundament1, Tube1, Rotor1, Wing1)
Windmill1.assemble()
print(Windmill1._fund_center)
print(Windmill1._fan_center)

(0, 0, -3.0)
(0, 21.0, 45.0)
